In [24]:
import h5py
import numpy as np
import os,sys
from scipy import stats
import random
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../models/20210219_MC_ensemble/')

# Rejection algorithm based on random walk
We created a tiny ensemble of 100 different SHEMAT-Suite and will use a rejection algorithm based on the Metropolis acceptance probability to get a posterior ensemble of models.  
The Metropolis acceptance probability is defined as:  

$$ \alpha(x_{t-1},z) = \begin{cases} min\big(\frac{p(z)}{p(x_{t-1})},1\big), & \text{if } p(x_{t-1}) > 0\\
1, & \text{if } p(x_{t-1}) = 0 \end{cases} $$  

A different approach would be to assess the missfit (as RMS error) of each realisation.  

$$ \alpha(x_{t-1},z) = \begin{cases} exp\big(-\frac{S(z) - S(x_{t-1}) }{u_T}\big), & \text{if } S(z) > S(x_{t-1})\\
1, & \text{otherwise }  \end{cases} $$  

We will use the second approach for now...also because we wrote it in the abstract.  
As discretization error, we take a value from Elison(2015), $u_{T-discr} = 0.7$ K  

Using Gauss error propagation, we assess a potential error for the realisations.  

$$ u_T = \sqrt{\big(\frac{\partial T}{\partial x_1}u_1 \big)^2 + ... + \big(\frac{\partial T}{\partial x_n}u_n \big)^2} $$

Literature sources for log-errors:
_The lower part of the disturbed log profile (below the cross-over point) was rotated to match these corrected tempera-tures. In the upper part of the profile, the same correction as for method A was applied. The quality of this correction method strongly depends on the correct calculation of the lowermost profile temperatures. According to Förster (2001), most of the corrected tem-peratures have errors of ± 3 to 5 K._ https://doi.org/10.1186/s40517-020-00181-w  


 _The effective accuracy of commercial temperature logs is ±0.5ºC (Blackwell and Spafford, 1987)._  http://www.sprensky.com/publishd/temper2.html  
 
 _More normal accuracies are +- 0.25 °C over 0-200 °C_ Keith Geothermal Energy lecture  
 
 For errors as a function of e.g. logging speed, measurement response time etc, look https://doi.org/10.1016/j.petrol.2020.107727

In [2]:
def fahrenheit_to_celsius(temp_fahrenheit, difference=False):
    if not difference:
        return (temp_fahrenheit - 32) * 5 / 9
    else:
        return temp_fahrenheit * 5 / 9

In [4]:
# define uT
T_error = 0.25 # temperature error tool accuracy
s_error = fahrenheit_to_celsius(1.25, difference=True) # sensor response time of 2 sec and 1 year after drilling
l_error = fahrenheit_to_celsius(1.25, difference=True) # logging speed of 20/ft after 1 year
d_error = 1.0 # estimated temperature error by discretization
#u_T = np.sqrt(T_error[0]**2 + T_error[1]**2 + T_error[2]**2 + T_error[3]**2 + d_error**2)
#u_T = np.sum(T_error**2)/4
u_T = np.sqrt(T_error**2 + s_error**2 + l_error**2 + d_error**2)
print(u_T)

1.4237296698599444


In [11]:
# load Simulation outputs. Those outputs get written by SHEMAT-Suite if runmode = 1
outp_path = '../models/20210219_MC_ensemble/'
diffs = np.loadtxt(outp_path+'PCT_MC_0_final.dat',skiprows=3,usecols=(8,),dtype=float)
for i in range(1,100):
    n = np.loadtxt(outp_path+f'PCT_MC_{i}_final.dat',skiprows=3,usecols=(8,),dtype=float)
    diffs=np.vstack([diffs,n])

In [15]:
# calculate RMSE of each realisation.
n = diffs.shape[1] # as we have 4 data points for temperature

diffs_sq = diffs**2
ssr = diffs_sq.sum(axis=1)
rmse = np.sqrt((diffs_sq.sum(axis=1)/n))

In [19]:
# this is a matrix with all vectors. First 96 columns are differences of the wells, then the column is the SSR, 
# final column is RMSE
tot_diffs = np.column_stack((diffs,ssr,rmse))
print(tot_diffs.shape)
# add index to the realizations
ind = np.array(range(100))
tot_diffs = np.column_stack((tot_diffs,ind))

(100, 98)


In [46]:
diffs[0,:] - diffs[72,:]

array([ 6.2329e-03,  6.0277e-03,  5.8208e-03,  5.6130e-03,  5.4050e-03,
        5.1973e-03,  4.9909e-03,  4.7861e-03,  4.5834e-03,  4.3831e-03,
        4.1859e-03,  3.9920e-03,  3.8018e-03,  3.6154e-03,  3.4330e-03,
        3.2543e-03,  3.0793e-03,  2.9083e-03,  2.7418e-03,  2.5801e-03,
        2.4235e-03,  2.2721e-03,  2.1259e-03,  1.9847e-03,  1.8479e-03,
        1.7139e-03,  1.5836e-03,  1.4595e-03,  1.3402e-03,  1.2253e-03,
        1.1150e-03,  1.0095e-03,  9.2400e-04,  8.3520e-04,  7.2900e-04,
        6.0380e-04,  4.5940e-04,  3.1810e-04,  1.8230e-04,  7.3870e-03,
        6.9158e-03,  6.4415e-03,  5.9689e-03,  5.5032e-03,  5.0492e-03,
        4.6110e-03,  4.1921e-03,  3.7946e-03,  3.4189e-03,  3.0628e-03,
        2.6704e-03,  2.3512e-03,  2.1164e-03,  1.8676e-03,  1.6065e-03,
        1.3312e-03,  1.0815e-03,  8.5250e-04,  6.3970e-04,  4.4760e-04,
        2.7110e-04, -2.1327e-02, -1.5971e-02, -1.1851e-02, -8.3670e-03,
       -5.8670e-03, -3.9690e-03, -2.5850e-03, -1.7440e-03, -1.25

## Rejection sampling
we now start with a random sample and go randomly through the pool, accepting and rejecting realizations.
The algorithm starts with one refrence sample `Ref`. Then, iteratively, samples (= realizations) get accepted, rejected based on their RMSE values. That is why we use the 6th column of `tot_diffs`. Alternatively, one could also just use the `rmse` array.

In [40]:
np.exp(-(tot_diffs[65,col] - Ref)/(u_T))

0.9984297438727661

In [79]:
# Chronological implemntation - start von 1 bis N 
# Can be used here, if samples generated are already in a random order and not correlated.
# That is usually the case with GemPy exports to SHEMAT-Suite.
col = 97
Ref = tot_diffs[0,col]
accept = []
P = []
k=0
for i in range(1,100):
    if tot_diffs[i,col] < Ref:
        Ref = tot_diffs[i,col]
        accept.append(i)
        
    elif random.random() < np.exp(-(tot_diffs[i,col] - Ref)/(u_T)):
        P.append(np.exp(-(tot_diffs[i,col] - Ref)/(u_T)))
        Ref = tot_diffs[i,col]
        accept.append(i)
        k += 1
print(len(accept))
print(accept)

99
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


And we see, temperature data is not sensitive to changes in the PCT-depth.  

But what if we also treat the thermal conductivity as an uncertain parameter?